In [ ]:
!pip install praw
!pip install psaw
import re
import praw
import pandas as pd
import numpy as np
import nltk
import warnings
nltk.download('stopwords')
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from psaw import PushshiftAPI

warnings.filterwarnings('ignore')
api = PushshiftAPI()
reddit = praw.Reddit(client_id='niNh_Elui1OusA', client_secret='gSMcp4CW3nHLolSLmGkiI6Dq_fw', user_agent='Scraping')
api = PushshiftAPI(reddit)

In [ ]:
def clean(text):

    no_cite = re.sub('>.*','',text)

    no_url = re.sub(r"http\S+", '', no_cite)

    review_text = BeautifulSoup(no_url).get_text()

    #Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    
    #Convert to lower case and split
    words = letters_only.lower().split()
    
    #Covert stop words to a set
    stops = set(stopwords.words("english"))
    
    #Remove stopwords
    meaningful_words = [w for w in words if not w in stops]
    
    #Join the words back into one string separated by space
    clean_text = " ".join(meaningful_words)

    return clean_text

In [ ]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_vectors(*strs):
    text = [clean(t) for t in strs]
    vectorizer = CountVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()
def get_consine_sim(*strs):
    vectors = [t for t in get_vectors(*strs)]
    return cosine_similarity(vectors)

# Scrape raw data from Reddit with PRAW and PSAW

In [ ]:
with open("data/Examples.txt","r") as f:
    lines = f.readlines()

verified = [line.strip() for line in lines if line!='\n']

In [ ]:
gen1 = api.search_comments(q=' live ',subreddit='Atlanta')

comments = []
comments_body = []
comments_author = []

for i in gen1:

    c_id = str(i)
    comment_entity=reddit.comment(c_id)
    c_body = comment_entity.body
    c_author = str(comment_entity.author)
    verified_rest = list(set(verified) - set([i]))
    score = max([np.mean([get_consine_sim(text,j)[1,0] for j in verified_rest]) for text in c_body.split('\n')])
    
    if score>0.075:
        comments.append(c_id)
        comments_body.append(c_body)
        comments_author.append(c_author)
    if len(comments)>1000000:
        break

In [12]:
database = pd.DataFrame({'ID':comments,'body':comments_body,'author':comments_author})

# Identify race identity of authors

In [ ]:
author_list = list(set(database['author']))

In [ ]:
keys_w=["I\'m a white","I am a white","I\'m white","I am white","As a white"]
keys_b=["I\'m a black","I am a black","I\'m black","I am black","As a black"]
race={}
for author in author_list:
    race[author]=0
    for key_w in keys_w:
        comments = api.search_comments(author=author, q=key_w)
        for comment_id in comments:
            comment_body = reddit.comment(str(comment_id))
            if comment_body:
                race[author]+=-1 
    for key_b in keys_b:
        comments = api.search_comments(author=author, q=key_w)
        for comment_id in comments:
            comment_body = reddit.comment(str(comment_id))
            if comment_body:
                race[author]+=1  

In [ ]:
race_label=[]
for author in database['author']:
    if race[author]==1:
        race_label.append('black')
    elif race[author]==-1:
        race_label.append('white')
    else:
        race_label.append(None)

In [ ]:
database['race']=race_label
databas = database.dropna()
database.to_csv('data/complete_data.csv')